In [1]:
import pandas as pd
import requests 
import threading
from tqdm import tqdm 
import json

In [6]:
base_url = 'http://119.188.67.99:6007'

def _test_api_call(operation):
    this_url = base_url + '/' + operation[0]
    mothods = operation[1]
    args = operation[2]
    if args is not None:
        for arg in args:
            this_url += ('/' + str(arg))
    data_in_json = operation[3]

    # print(f'url: {this_url}')
    #     print(f'json: {data_in_json}')

    if mothods == 'POST':
        response = requests.post(this_url, json=data_in_json)
    elif mothods == 'GET':
        print(args)
        response = requests.get(this_url)
    # elif mothods == 'PUT':
        # response = requests.put(the_url, json=data_in_json)
    else:
        raise ValueError('Unsupported HTTP method')
    
    # 检查响应状态码
    # print(response.status_code)
    # print(response.json())
    # print()
    return response.json()

In [2]:
infered_data_path = '/data/lj/llm/dataset/iter3/raw_data.csv'
infered_data = pd.read_csv(infered_data_path)
infered_data.head(2)

,q,que,ai_answer,actual_send_answer,new_refer,kb_ids
0,一包肠依做多少肉,"[{""role"":""assistant"",""content"":""很高兴为您服务,请问有什么可...","亲,一包肠衣可以做8-10斤肉哦~","猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,...","['问题：""亲一袋肠衣能罐多少斤肉"", 答案：""猪肠衣一小包，罐大约4斤肉""', '问题：""...","22, 23"
1,"我刚才试了一下,只能买一件呀","[{""role"":""user"",""content"":""亲在吗""},{""role"":""user...","亲,您可以在订单中分别选择一袋肠衣和五香调料、麻辣调料,然后一起下单哦。",分开拍下 拍完给您尽量一起发货,"['问题：""我刚拍下了这款不是一包一包的吗"", 答案：""我们是两种包装 随机发的哦""', '...","22, 23"


In [3]:

kb_ids = infered_data['kb_ids'].tolist()
ques = infered_data['que'].tolist()
qs = [eval(q)[-1]['content'] for q in ques]

In [4]:

def get_model_answer(eval_data,model_name:str):
    kb_ids = eval_data['kb_ids'].tolist()
    kb_ids_split = [kb_id.split(', ') for kb_id in kb_ids]
    kb_ids_strip = [[item.strip() for item in sublist] for sublist in kb_ids_split]
    
    ques = eval_data['que'].tolist()
    ai_results = []

    for i,(que,kbids) in enumerate(zip(ques,kb_ids_strip)):
        ai_res = _test_api_call(('ask', 'POST', None, {'kb_ids': kbids, 'messages': eval(que)}))
        ai_results.append(ai_res)
        if i %10 == 0:
            print(i+1,eval(que)[-1]['content'],ai_res['answer'])
        
    assert len(eval_data) == len(ai_results)
    eval_data[model_name] = [res['answer'] for res in ai_results] 


In [7]:
# 获得召回信息 平时用不到
def get_refer(eval_data,model_name:str):
    kb_ids = eval_data['kb_ids'].tolist()
    kb_ids_split = [kb_id.split(',') for kb_id in kb_ids]
    kb_ids_strip = [[item.strip() for item in sublist] for sublist in kb_ids_split]
    
    ques = eval_data['que'].tolist()
    ai_results = []

    for i,(que,kbids) in enumerate(zip(ques,kb_ids_strip)):
        
        
        ai_res = _test_api_call(('test_kb_retrieval', 'POST', None, {'kb_ids': kbids, 'query': eval(que)[-1]['content']}))
        
        ref = [ref['context'] for ref in ai_res['data']['knowledges']]
        
        # if eval(que)[-1]['content'] == '有人吗':
        #     print('q',eval(que)[-1]['content'],'kb_ids',kbids)
        #     print(ref)
            
        
        ai_results.append(ref)
        if i %10 == 0:
            print(i+1,eval(que)[-1]['content'],ref)
        
    assert len(eval_data) == len(ai_results)
    # eval_data[model_name] = ai_results
    return ai_results
    
# 如果是一个
# ai_res = _test_api_call(('test_kb_retrieval', 'POST', None, {'kb_ids': ['22','23'], 'query': "你好这个是粗的还是细的"}))
# print(ai_res)
model_name = "new_refer"
if model_name in infered_data.columns:
    # 存在则删除 'B' 列
    infered_data.drop(columns=[model_name], inplace=True)

refer_list = get_refer(infered_data,model_name)
infered_data[model_name] = refer_list
infered_data.to_csv('/data/lj/llm/dataset/iter3/raw_data_new.csv')

infered_data.head(1)

1 一包肠依做多少肉 ['问题："一包肠衣灌几斤肉啊", 答案："亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌8-10斤肉，根据个人手法会有所不同哦。"', '问题："一袋肠衣能做几斤肉", 答案："猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，口径3.8-4.0左右，灌大约8-10斤肉。"', '问题："100g肠衣能灌多少肉", 答案："亲 不清楚哦"', '问题："一包猪肠子适合几斤肉？", 答案："猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，口径3.8-4.0左右，灌大约8-10斤肉。"', '问题："这一包肠衣可以灌多少斤肉？", 答案："猪肠衣五斤一包4-5米左右，口径3.8-4.0左右，灌大约4斤肉 10斤一包8-10米左右，口径3.8-4.0左右，灌大约8-10斤肉。"']
11 好了不跟你计较可以多些都结了。 ['问题："可以，本来就那么点钱纠结也无意义", 答案："亲，确认收货以后联系我们帮您处理哈"', '问题："你现在退多多钱包2元，到时候收后就不用麻烦了", 答案："亲 原路返回的哦"', '问题："一共三包现在打开一包了也可以退吗？", 答案："亲可以退剩下的哦"', '问题："你们不给也行，给退钱吧。", 答案："商品寄回有专人处理"', '问题："我就半杯水加了四勺", 答案："亲，你可以多放一些哦"']
21 血糖高化合物高不能吃 ['问题："哦，血糖高多人能吃不", 答案："建议咨询医生哈"', '问题："高血糖可以吃太甜了不能吃", 答案："实在抱歉给您带来不好的体验，如果您实在不满意的话您可以申请退货退款，商品退回，有专人处理的哈。"', '问题："亲，糖尿病人吃，血糖升高？", 答案："零添加蔗糖哈"', '问题："我们都是糖尿病人不能吃甜的", 答案："亲 可以申请退货退款哦"', '问题："孩子糖尿病，不能吃带糖的", 答案："亲 建议咨询医生哦"']
31 我要巧克力和草莓行吗？ ['问题："我要巧克力和草莓行吗？", 答案："可以的亲这边帮您备注"', '问题："草莓，巧克力", 答案："已备注"', '问题："亲，我想要草莓和芒果味可以吗", 答案："亲，下单随意搭配，联系客服备注哈"', '问题："亲，我想要草莓和

,q,que,ai_answer,actual_send_answer,kb_ids,new_refer
0,一包肠依做多少肉,"[{""role"":""assistant"",""content"":""很高兴为您服务,请问有什么可...","亲,一包肠衣可以做8-10斤肉哦~","猪肠衣五斤一包4-5米左右,口径3.8-4.0左右,灌大约4斤肉 10斤一包8-10米左右,...","22, 23","[问题：""一包肠衣灌几斤肉啊"", 答案：""亲，5斤一包肠衣可以灌5斤左右肉，10斤装的可以灌..."
